In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("./MEX/mex_distance_matrix.csv")
df = df.drop(['Unnamed: 0', 'shapeID'], axis = 1)
df.head(10)

,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B9,MEX-ADM2-1590546715-B10,...,MEX-ADM2-1590546715-B2449,MEX-ADM2-1590546715-B2450,MEX-ADM2-1590546715-B2451,MEX-ADM2-1590546715-B2452,MEX-ADM2-1590546715-B2453,MEX-ADM2-1590546715-B2454,MEX-ADM2-1590546715-B2455,MEX-ADM2-1590546715-B2456,MEX-ADM2-1590546715-B2457,MEX-ADM2-1590546715-B2458
0,0.000000,0.000000,0.055894,0.135329,0.083422,0.000000,0.000000,0.000000,0.000000,0.373148,...,3.590395,3.301435,3.538930,3.283451,3.393920,3.570205,3.325766,3.747767,3.637447,6.987838
1,0.000000,0.000000,0.056499,0.066500,0.000000,0.025508,0.000000,0.000000,0.037331,0.271604,...,3.789347,3.477868,3.725380,3.497023,3.605621,3.772707,3.539370,3.978910,3.868678,7.211869
2,0.055894,0.056499,0.000000,0.000000,0.057884,0.000000,0.134585,0.000000,0.220951,0.231698,...,3.981246,3.672690,3.919154,3.687828,3.796559,3.963738,3.730176,4.167426,4.057289,7.401799
3,0.135329,0.066500,0.000000,0.000000,0.000000,0.083889,0.086895,0.000000,0.261674,0.000000,...,3.947168,3.622045,3.875257,3.677237,3.782247,3.933351,3.719507,4.181901,4.071220,7.400776
4,0.083422,0.000000,0.057884,0.000000,0.000000,0.141160,0.000000,0.000000,0.113149,0.000000,...,3.822413,3.504149,3.754489,3.545388,3.651453,3.807358,3.587697,4.044269,3.933669,7.267254
5,0.000000,0.025508,0.000000,0.083889,0.141160,0.000000,0.056798,0.000000,0.120956,0.346029,...,3.884624,3.585746,3.828888,3.582824,3.692719,3.865501,3.625155,4.054205,3.944155,7.291909
6,0.000000,0.000000,0.134585,0.086895,0.000000,0.056798,0.000000,0.017952,0.000000,0.000000,...,3.548168,3.226176,3.477774,3.278407,3.383091,3.534083,3.320660,3.776871,3.666266,7.000347
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017952,0.000000,0.101724,0.166607,...,3.849803,3.543507,3.788903,3.558840,3.667094,3.832473,3.601187,4.042813,3.932535,7.274808
8,0.000000,0.037331,0.220951,0.261674,0.113149,0.120956,0.000000,0.101724,0.000000,0.358514,...,3.513912,3.227151,3.463549,3.205517,3.316146,3.493420,3.247826,3.673833,3.563767,6.912072
9,0.373148,0.271604,0.231698,0.000000,0.000000,0.346029,0.000000,0.166607,0.358514,0.000000,...,3.873126,3.528537,3.789451,3.625471,3.726908,3.859831,3.667740,4.151645,4.039856,7.350714


In [5]:
df.shape

(2458, 2458)

In [6]:
def get_mins(x, num, cur_id):
    
    # Delete the current polygon from the list so we do no include it in our list of closest polgyons, then cast the dict back to a list
    x_dict = dict(x)
    del x_dict[cur_id]
    x = list(x_dict.values())
    
    # sort the distances from lowest to highest
    x_sorted = sorted(x)
    
    # If there are more polgyons that are a distance of 0 than the specified number, reutrn ALL of the indices of polygons that have a distance of zero
    if x.count(0) > num:
        zeros = x_sorted[0:x.count(0)]
        x_array = np.array(x)
        return np.where(x_array == 0)[0]
    
    # Otherwise, return the indices of the num closest polygons
    else:
        minimums = x_sorted[0:num]
        x_array = np.array(x)
        indices = [list(np.where(x_array == i)[0]) for i in minimums]
        inidces = [item for sublist in indices for item in sublist]
        res = []
        [res.append(x) for x in inidces if x not in res]
        return res

In [7]:
min_list = []
num = 8
c = 0


for col, row in df.iterrows():
    
    # Get the indices of the num closest polygons to the currect one
    mins = get_mins(row, num, df.columns[c])
    print(mins)
    
    # The get_min algo does not include the current polygon so drop that for the list of column we are going to grab from using the min indices
    ref_cols = [i for i in df.columns if i != df.columns[c]]
    
    # Get the shapeID's of the num closest polygons, append them to the overall list and increment c
    min_ids = [ref_cols[i] for i in mins]
    min_list.append(min_ids)
    c += 1
    

# Make the final data frame
min_df = pd.DataFrame(min_list)
min_df = min_df.drop([9,])
min_df.columns = ['closest' + str(i) for i in range(0, num)]
min_df = min_df.reset_index()
min_df['index'] = df.columns#[0:5]
min_df = min_df.rename(columns = {'index':'shapeID'})
min_df.head()

 2032 2118 2138 2145 2146 2148 2162]
[1995, 2006, 2044, 2061, 2071, 2094, 2102, 2132]
[1961 1996 2007 2068 2076 2093 2110 2147 2157]
[1161, 1422, 2067, 1438, 886, 1078, 1968, 1424]
[1959, 2002, 2006, 2051, 2061, 2030, 2143, 1969]
[456, 479, 1496, 1529, 2018, 2028, 2041, 2078]
[1988, 2022, 2071, 2083, 2123, 2126, 2164, 2040]
[1959, 1969, 2049, 2077, 2142, 2006, 2143, 2002]
[1009, 1096, 1410, 1654, 1657, 1973, 2031, 2138]
[1691 1724 1734 1865 1874 1876 1891 2027 2055 2062 2122 2152]
[1974, 2019, 2021, 2060, 2121, 2135, 1593, 1970]
[1966 2010 2013 2025 2027 2054 2062 2116 2165]
[1967, 1989, 2003, 2042, 2064, 2065, 2029, 2143]
[1531 1998 2011 2024 2075 2076 2103 2111 2145]
[1529, 2020, 2078, 1489, 1496, 2001, 2050, 2041]
[1955, 2029, 2077, 2136, 2142, 2037, 1969, 1983]
[1593, 1601, 2021, 2055, 2135, 2038, 1613, 1539]
[1959, 2006, 2030, 2044, 2046, 2049, 2067, 2094]
[2054, 2056, 2122, 2165, 1876, 2025, 1868, 1874]
[2001, 2020, 2038, 2017, 2140, 2088, 2078, 2072]
[1955, 1979, 1983, 1989, 202

ValueError: Length mismatch: Expected axis has 22 elements, new values have 8 elements

In [12]:
# Make the final data frame
min_df = pd.DataFrame(min_list)
min_df = min_df.drop([8,9,10,11,12,13,14,15,16,17,18,19,20,21], axis = 1)
min_df.columns = ['closest' + str(i) for i in range(0, num)]
min_df = min_df.reset_index()
min_df['index'] = df.columns#[0:5]
min_df = min_df.rename(columns = {'index':'shapeID'})
min_df.head()

,shapeID,closest0,closest1,closest2,closest3,closest4,closest5,closest6,closest7
0,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B9,MEX-ADM2-1590546715-B613,MEX-ADM2-1590546715-B2301,MEX-ADM2-1590546715-B2302
1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B9,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B4
2,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B11,MEX-ADM2-1590546715-B2274,MEX-ADM2-1590546715-B2294,MEX-ADM2-1590546715-B2326,MEX-ADM2-1590546715-B2301
3,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B10,MEX-ADM2-1590546715-B2325,MEX-ADM2-1590546715-B2326,MEX-ADM2-1590546715-B2274,MEX-ADM2-1590546715-B2
4,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B10,MEX-ADM2-1590546715-B615,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B1


In [24]:
# # Make the final data frame
# min_df = pd.DataFrame(min_list)
# min_df = min_df.drop([8,9,10,11,12,13,14,15,16,17,18,19,20,21], axis = 1)
# min_df.columns = ['closest' + str(i) for i in range(0, num)]
# min_df = min_df.reset_index()
# min_df['index'] = df.columns#[0:5]
# min_df = min_df.rename(columns = {'index':'shapeID'})
# min_df.head()

,shapeID,closest0,closest1,closest2,closest3,closest4,closest5,closest6,closest7
0,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B9,MEX-ADM2-1590546715-B613,MEX-ADM2-1590546715-B2301,MEX-ADM2-1590546715-B2302
1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B9,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B4
2,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B11,MEX-ADM2-1590546715-B2274,MEX-ADM2-1590546715-B2294,MEX-ADM2-1590546715-B2326,MEX-ADM2-1590546715-B2301
3,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B10,MEX-ADM2-1590546715-B2325,MEX-ADM2-1590546715-B2326,MEX-ADM2-1590546715-B2274,MEX-ADM2-1590546715-B2
4,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B10,MEX-ADM2-1590546715-B615,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B1


In [13]:
min_df.to_csv("./MEX/closest_polygons.csv", index = False)

In [14]:
min_df.head()

,shapeID,closest0,closest1,closest2,closest3,closest4,closest5,closest6,closest7
0,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B9,MEX-ADM2-1590546715-B613,MEX-ADM2-1590546715-B2301,MEX-ADM2-1590546715-B2302
1,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B1,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B9,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B4
2,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B6,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B11,MEX-ADM2-1590546715-B2274,MEX-ADM2-1590546715-B2294,MEX-ADM2-1590546715-B2326,MEX-ADM2-1590546715-B2301
3,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B10,MEX-ADM2-1590546715-B2325,MEX-ADM2-1590546715-B2326,MEX-ADM2-1590546715-B2274,MEX-ADM2-1590546715-B2
4,MEX-ADM2-1590546715-B5,MEX-ADM2-1590546715-B2,MEX-ADM2-1590546715-B4,MEX-ADM2-1590546715-B7,MEX-ADM2-1590546715-B8,MEX-ADM2-1590546715-B10,MEX-ADM2-1590546715-B615,MEX-ADM2-1590546715-B3,MEX-ADM2-1590546715-B1


In [15]:
min_df.shape

(2458, 9)